In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
import keras.layers.merge
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
# from keras.callbacks import EarlyStopping
import gensim
# import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
EMBEDDING_DIM = 500
MAX_VOCAB_SIZE = 19911
MAX_SEQUENCE_LENGTH = 85

#training params
batch_size = 256  
num_epochs = 20 

In [ ]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [ ]:
clean_train_comments = pd.read_csv("./corpus/tripadvisor/train_set.csv")
clean_train_comments['content'] = clean_train_comments['content'].astype('str')
clean_train_comments["tokens"] = clean_train_comments["content"].str.split()
clean_train_comments['sentiment'] = clean_train_comments['polarity'].apply(sentiment_label)
   
clean_train_comments.head()

In [ ]:
clean_test_comments = pd.read_csv("./corpus/tripadvisor/test_set.csv")
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments["tokens"] = clean_test_comments["content"].str.split()
clean_test_comments['sentiment'] = clean_test_comments['polarity'].apply(sentiment_label)

clean_test_comments.head()

In [ ]:
all_training_words = [word for tokens in clean_train_comments["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in clean_train_comments["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

In [ ]:
all_test_words = [word for tokens in clean_test_comments["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in clean_test_comments["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

In [ ]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(clean_train_comments["content"].tolist())
training_sequences = tokenizer.texts_to_sequences(clean_train_comments["content"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(clean_test_comments["content"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=True, extra_conv=False):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    #l_merge = Merge(mode='concat', concat_axis=1)(convs)
    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc']) 
    model.summary()
    return model

In [ ]:
y_tr = clean_train_comments['sentiment'].values
y_ts = clean_test_comments['sentiment'].values

In [ ]:
x_train = train_cnn_data
y_train = y_tr

x_test = test_cnn_data
y_test = y_ts

In [ ]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 1)

In [ ]:
hist = model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), batch_size=batch_size)

In [ ]:
# model.save('./model/yoon_kim/cnn_model_06.h5')  

In [ ]:
# model = load_model('./model/yoon_kim/cnn_model_06.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_predict = model.predict(test_cnn_data, batch_size=batch_size, verbose=1)
for i in range(len(y_predict)):
    y_predict[i][0] = round(y_predict[i][0])
print(classification_report(y_test, y_predict, labels = [0, 1], digits=8))

In [ ]:
#generate plots
# plt.figure()
# plt.plot(hist.history['loss'], lw=2.0, color='b', label='train')
# plt.plot(hist.history['val_loss'], lw=2.0, color='r', label='val')
# plt.title('CNN sentiment')
# plt.xlabel('Epochs')
# plt.ylabel('Cross-Entropy Loss')
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(hist.history['acc'], lw=2.0, color='b', label='train')
# plt.plot(hist.history['val_acc'], lw=2.0, color='r', label='val')
# plt.title('CNN sentiment')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend(loc='upper left')
# plt.show()